In [6]:
from functools import wraps
import contextlib
from typing import Optional
from datetime import date
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlmodel import SQLModel, Field, Relationship
from typing import List
import os

In [7]:
def db_engine():
    return create_engine("sqlite:///newswire.db")

def get_db_session():
    engine = db_engine()
    return Session(engine)

class DatabaseSession:
    def __init__(self):
        self._session = None

    @property
    def session(self):
        return self._session

    def set_session(self, session):
        self._session = session

def db_session(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        db = DatabaseSession()
        with get_db_session() as session:
            db.set_session(session)
            result = func(db, *args, **kwargs)
            session.commit()
        return result
    return wrapper

In [8]:
class Source(SQLModel, table=True):
    id: int = Field(primary_key=True)
    name: str = Field(nullable=False)
    newswires: List["Newswire"] = Relationship(back_populates="source")

class Newswire(SQLModel, table=True):
    id: int = Field(primary_key=True)
    title: str = Field(nullable=False)
    article_md: Optional[str]
    link: str = Field(nullable=False)
    article_sum_md: Optional[str]
    source_text: Optional[str]
    llm_params: Optional[str]
    date_created: date = Field(default=date.today(), nullable=False)
    date_updated: date = Field(default=date.today(), nullable=False)
    article_date: Optional[date]
    tone: Optional[str]
    emotional_impact: Optional[str]
    word_count: Optional[int]
    source_id: int = Field(foreign_key="source.id")
    source: Source = Relationship(back_populates="newswires")
    related_links: List["RelatedLink"] = Relationship(back_populates="newswire")

class RelatedLink(SQLModel, table=True):
    id: int = Field(primary_key=True)
    url: str = Field(nullable=False)
    description: Optional[str]
    newswire_id: int = Field(foreign_key="newswire.id")
    newswire: Newswire = Relationship(back_populates="related_links")

# with SessionLocal() as session:
#     SQLModel.metadata.drop_all(bind=session)

In [9]:
SQLModel.metadata.create_all(get_db_session())

AttributeError: 'Session' object has no attribute '_run_ddl_visitor'

In [ ]:
@db_session
def create_news_wires(db, num_newswire_examples=5):
        for i in range(num_newswire_examples):
            src = Source(name="Howdy Example")
            nw = Newswire(title=f"Newswire Example {i+1}", link=f"https://example.com/{i}", source=src)
            db.session.add(nw)
        
create_news_wires(5)


In [10]:
from faker import Faker

fake = Faker()

@db_session
def create_news_wires(db, num_newswire_examples=5):
    for i in range(num_newswire_examples):
        source_name = fake.company() 
        src = Source(name=source_name) 
        
        nw_title = fake.catch_phrase()
        nw_link = fake.url()
        nw_article_md = fake.text()
        nw_article_sum_md = fake.paragraph()
        nw_source_text = fake.word()
        nw_llm_params = fake.word()
        nw_date_created = fake.date_this_year()
        nw_date_updated = fake.date_this_year()
        nw_article_date = fake.date_this_year()
        nw_tone = fake.random_element(elements=('Positive', 'Negative', 'Neutral'))
        nw_emotional_impact = fake.random_element(elements=('High', 'Medium', 'Low'))
        nw_word_count = fake.random_int(min=200, max=2000, step=1)
        
        nw = Newswire(title=nw_title, link=nw_link, article_md=nw_article_md, article_sum_md=nw_article_sum_md, 
                      source_text=nw_source_text, llm_params=nw_llm_params, date_created=nw_date_created, 
                      date_updated=nw_date_updated, article_date=nw_article_date, tone=nw_tone, 
                      emotional_impact=nw_emotional_impact, word_count=nw_word_count, source=src)
        
        db.session.add(nw)
        
create_news_wires(50) #creating 50 newswires instances with respective sources